In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold

In [ ]:
def get_stratified_from_oof(df, n_splits=5):
    cols_to_use = ['id', 'url_legal', 'license', 'excerpt', 'target', 'standard_error', 'oof']
    df = df.loc[:, cols_to_use]
    df['abs_error'] = abs(df['oof'] - df['target'])
    
    n_bins = int(np.floor(1 + np.log2(len(df))))
    df['bin'] = pd.cut(df.abs_error, n_bins, labels=False)
    
    skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
    df['fold'] = np.nan

    for fold, (idx_tr, idx_val) in enumerate(skf.split(df.id, y=df.bin)):
        df.loc[idx_val, 'fold'] = fold

    df['fold'] = df['fold'].astype('int8')
    return df

In [ ]:
TRAIN = '../input/clrp-robertabase-from-colab/oof_df_for_train.csv'
SEED = 567

df = pd.read_csv(TRAIN)
df = df.loc[df.standard_error!=0].reset_index(drop=True)
df = get_stratified_from_oof(df)

df.to_csv("train_oof_stratified.csv")